# Note: In order for XGBoost to work, you need to pass all the inputs.
### In other words, please make sure that you aren't using the simple NN when you run XGBoost

In [1]:
import os
import sys

import pytorch_lightning as pl
from omegaconf import DictConfig, OmegaConf
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import random_split
from torch_geometric.data import DataLoader

from spatial.merfish_dataset import MerfishDataset
from spatial.models.monet_ae import MonetAutoencoder2D, TrivialAutoencoder
from spatial.train import train
from spatial.predict import test

In [2]:
# equivalent to spatial

import hydra
from hydra.experimental import compose, initialize

with initialize(config_path="../config"):
    cfg_from_terminal = compose(config_name="config")
    output = test(cfg_from_terminal)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory /home/roko/spatial//output/lightning_logs/checkpoints/MonetAutoencoder2D exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 1.4626601934432983}
--------------------------------------------------------------------------------


# MAE Imaging

In [3]:
import matplotlib.pyplot as plt

trainer, l1_losses, inputs, gene_expressions, celltypes = output

We can see in the source code that the cell types are listed as follows:

    cell_types = [
        "Ambiguous",
        "Astrocyte",
        "Endothelial 1",
        "Endothelial 2",
        "Endothelial 3",
        "Ependymal",
        "Excitatory",
        "Inhibitory",
        "Microglia",
        "OD Immature 1",
        "OD Immature 2",
        "OD Mature 1",
        "OD Mature 2",
        "OD Mature 3",
        "OD Mature 4",
        "Pericytes",
    ]
    
So, we can keep only the cell types the MESSI uses to evaluate their performance. These would be [1, 2, 6, 7, 8, 9, 10, 11]

#### For a single sample.

# FIX THIS TO BE BOXPLOT OVER ALL EXPRESSIONS

In [4]:
# plt.boxplot(L1_losses.reshape(1,-1))

In [5]:
# plt.boxplot(L1_losses.reshape(1,-1)[:,:])

The code below simply identifies the response and non-response genes.

In [6]:
# read in merfish dataset and get columns names
import pandas as pd

# get relevant data stuff
df_file = pd.ExcelFile("~/spatial/data/messi.xlsx")
messi_df = pd.read_excel(df_file, "All.Pairs")
merfish_df = pd.read_csv("~/spatial/data/raw/merfish.csv")

# these are the 13 ligands or receptors found in MESSI
non_response_genes = ['Cbln1', 'Cxcl14', 'Crhbp', 'Gabra1', 'Cbln2', 'Gpr165', 
                      'Glra3', 'Gabrg1', 'Adora2a', 'Vgf', 'Scg2', 'Cartpt',
                      'Tac2']
# this list stores the control genes aka "Blank_{int}"
blank_genes = []
# ligands and receptor indexes in MERFISH
non_response_indeces = [list(merfish_df.columns).index(gene)-10 for gene in non_response_genes]
all_pairs_columns = [
    "Ligand.ApprovedSymbol",
    "Receptor.ApprovedSymbol",
]

# for column name in the column names above
for column in all_pairs_columns:
    for gene in merfish_df.columns:
        if (
            gene.upper() in list(messi_df[column])
            and gene.upper() not in non_response_genes
        ):
            non_response_genes.append(gene)
            non_response_indeces.append(list(merfish_df.columns).index(gene)-10)
        if gene[:5] == "Blank" and gene not in blank_genes:
            blank_genes.append(gene)
            non_response_indeces.append(list(merfish_df.columns).index(gene)-10)
print(non_response_genes)
print(
    "There are "
    + str(len(non_response_genes))
    + " genes recognized as either ligands or receptors (including new ones)."
)

print(
    "There are "
    + str(len(blank_genes))
    + " blank genes."
)

print(
    "There are "
    + str(160 - len(blank_genes) - len(non_response_genes)) # -5 is because of the blanks
    + " genes that are treated as response variables."
)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


['Cbln1', 'Cxcl14', 'Crhbp', 'Gabra1', 'Cbln2', 'Gpr165', 'Glra3', 'Gabrg1', 'Adora2a', 'Vgf', 'Scg2', 'Cartpt', 'Tac2', 'Bdnf', 'Bmp7', 'Cyr61', 'Fn1', 'Fst', 'Gad1', 'Ntng1', 'Pnoc', 'Selplg', 'Sema3c', 'Sema4d', 'Serpine1', 'Adcyap1', 'Cck', 'Crh', 'Gal', 'Gnrh1', 'Nts', 'Oxt', 'Penk', 'Sst', 'Tac1', 'Trh', 'Ucn3', 'Avpr1a', 'Avpr2', 'Brs3', 'Calcr', 'Cckar', 'Cckbr', 'Crhr1', 'Crhr2', 'Galr1', 'Galr2', 'Grpr', 'Htr2c', 'Igf1r', 'Igf2r', 'Kiss1r', 'Lepr', 'Lpar1', 'Mc4r', 'Npy1r', 'Npy2r', 'Ntsr1', 'Oprd1', 'Oprk1', 'Oprl1', 'Oxtr', 'Pdgfra', 'Prlr', 'Ramp3', 'Rxfp1', 'Slc17a7', 'Slc18a2', 'Tacr1', 'Tacr3', 'Trhr']
There are 71 genes recognized as either ligands or receptors (including new ones).
There are 5 blank genes.
There are 84 genes that are treated as response variables.


The code below generates some MAE values separated by response status.

In [7]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

print(len(inputs), len(gene_expressions))

non_response_inputs = torch.index_select(inputs, 1, torch.tensor(non_response_indeces))
response_inputs = torch.index_select(inputs, 1, torch.tensor(list(set(range(160)) - set(non_response_indeces))))
print("Initial Response Expression Mean: " + str(response_inputs.mean().item()))
print("Initial Non-Response Expression Mean: " + str(non_response_inputs.mean().item()))

non_response_expressions =  torch.index_select(gene_expressions, 1, torch.tensor(non_response_indeces))
response_expressions = torch.index_select(gene_expressions, 1, torch.tensor(list(set(range(160)) - set(non_response_indeces))))
print("AE Response Expression Mean: " + str(response_expressions.mean().item()))
print("AE Non-Response Expression Mean: " + str(non_response_expressions.mean().item()))

mean_MAE_L1 = F.l1_loss(inputs.mean(axis=0), gene_expressions.mean(axis=0)).item()

print("Average L1 Loss for Mean " + str(mean_MAE_L1))

### L1 Losses 
# response_gene_boxplot = abs(inputs.mean(axis=0) - gene_expressions.mean(axis=0))
# print(pd.DataFrame(response_gene_boxplot).describe())
# plt.boxplot(response_gene_boxplot.reshape(1,-1))
# _ = plt.show()

mean_MAE_response_L1 = F.l1_loss(response_inputs.mean(axis=0), response_expressions.mean(axis=0))
print("Average L1 Loss for Mean among Response Genes: " + str(mean_MAE_response_L1.item()))

mean_MAE_non_response_L1 = F.l1_loss(non_response_inputs.mean(axis=0), non_response_expressions.mean(axis=0))
print("Average L1 Loss for Mean among Non-Response Genes: " + str(mean_MAE_non_response_L1.item()))

173509 173509
Initial Response Expression Mean: 1.7898054122924805
Initial Non-Response Expression Mean: 1.3080828189849854
AE Response Expression Mean: 0.1979086995124817
AE Non-Response Expression Mean: 0.1586369425058365
Average L1 Loss for Mean 1.3821319341659546
Average L1 Loss for Mean among Response Genes: 1.5918967723846436
Average L1 Loss for Mean among Non-Response Genes: 1.1502866744995117


Now, we need to do the same thing, but separate the celltypes we care about.

In [8]:
import numpy as np
np.unique(celltypes[:,0])

array([3., 4., 5.], dtype=float32)

In [9]:
animals = [animal for animal, animal_id in enumerate(celltypes[:,0]) if animal_id in [3,4,5]]
cells = [cell for cell, celltype in enumerate(celltypes[:,1]) if celltype in [1, 2, 6, 7, 8, 9, 10, 11]]
print(len(animals), len(cells))

173509 117372


In [10]:
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt

# filtering the inputs this time
filtered_inputs = torch.index_select(inputs, 0, torch.tensor(list(set(cells) | set(animals))))
filtered_gene_expressions = torch.index_select(gene_expressions, 0, torch.tensor(list(set(cells) | set(animals))))

non_response_inputs = torch.index_select(filtered_inputs, 1, torch.tensor(non_response_indeces))
response_inputs = torch.index_select(filtered_inputs, 1, torch.tensor(list(set(range(160)) - set(non_response_indeces))))
print("Initial Response Expression Mean: " + str(response_inputs.mean().item()))
print("Initial Non-Response Expression Mean: " + str(non_response_inputs.mean().item()))

non_response_expressions =  torch.index_select(filtered_gene_expressions, 1, torch.tensor(non_response_indeces))
response_expressions = torch.index_select(filtered_gene_expressions, 1, torch.tensor(list(set(range(160)) - set(non_response_indeces))))
print("AE Response Expression Mean: " + str(response_expressions.mean().item()))
print("AE Non-Response Expression Mean: " + str(non_response_expressions.mean().item()))

mean_MAE_L1 = F.l1_loss(filtered_inputs.mean(axis=0), filtered_gene_expressions.mean(axis=0)).item()

print("Average L1 Loss for Mean " + str(mean_MAE_L1))

# response_gene_boxplot = abs(filtered_inputs.mean(axis=0) - filtered_gene_expressions.mean(axis=0))
# print(pd.DataFrame(response_gene_boxplot).describe())
# plt.boxplot(response_gene_boxplot.reshape(1,-1))
# _ = plt.show()

mean_MAE_response_L1 = F.l1_loss(response_inputs.mean(axis=0), response_expressions.mean(axis=0))
print("Average L1 Loss for Mean among Response Genes: " + str(mean_MAE_response_L1.item()))

mean_MAE_non_response_L1 = F.l1_loss(non_response_inputs.mean(axis=0), non_response_expressions.mean(axis=0))
print("Average L1 Loss for Mean among Non-Response Genes: " + str(mean_MAE_non_response_L1.item()))

Initial Response Expression Mean: 1.7898054122924805
Initial Non-Response Expression Mean: 1.3080828189849854
AE Response Expression Mean: 1.4492615461349487
AE Non-Response Expression Mean: 1.0914959907531738
Average L1 Loss for Mean 0.42155900597572327
Average L1 Loss for Mean among Response Genes: 0.49769583344459534
Average L1 Loss for Mean among Non-Response Genes: 0.3374076783657074


# Checklist of Things To Do:

- [x] Compare runs on tensorboard.
- [x] Ensure that outputs for every test example get saved (i.e. we should have a list of 31 tensors.) <b>No need. The tensor gets concatenated. We could create a tensor of tensors, but idk how helpful that is.</b>
- [x] Understand which anid, bregma correspond to which mouse and figure out how MESSI dealt with this. <b>Most animals have multiples bregmas and MESSI uses several animals (16-19) in their demo. </b>
- [x] Filter out outputs based on their respective celltype values (y).
- [ ] Implement XGBoost on the 8 celltypes with pip install xgboost.
- [x] Make batch size hydra parameter.
- [ ] Try multi-batch run with appropriate batch size and n_neighbors.
- [x] Change logger to only log MAE.
- [x] Add nbstripout to poetry.
- [x] Figure out how to label tensorboard folders.
- [x] Fix CUDA_VISIBLE_DEVICES <b>This is "fixed" but setting visible devices to more than 1 makes it "used" as a process even if a process doesn't get set to a GPU. Does being visible open up a process?</b>
- [x] Code up a basic NN that learns a single mean expression level for all genes.
- [ ] Hold out exact animals in testing batches.
- [ ] Run the actual MESSI Notebook
- [ ] Explore different neighborhood methodologies.
    - [ ] Nearest Neighbor Graph
    - [ ] Dulaney Trinagulation
    - [ ] Gabriel Graph
    - [ ] Urquhart Graph (this will be a very good comparison to Dulaney Triangulation) <b>Not a priority, because more training should take care of this.</b> 
- [ ] Edge weights for MONET based on cell distances.

In [11]:
import xgboost as xgb

In [12]:
inputs.numpy().shape

(173509, 160)

In [13]:
celltypes.shape

torch.Size([173509, 2])

#### The code below simply shows an example of classification with XGBoost.

In [14]:
# read in data
data = inputs.numpy()
labels = celltypes[:,1].numpy()
dtrain = xgb.DMatrix(data=data, label=labels)
dtest = xgb.DMatrix(gene_expressions.numpy())

# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'multi:softmax', 'num_class': 16}
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

/home/roko/.cache/pypoetry/virtualenvs/spatial-G_n0JvVf-py3.8/lib/python3.8/site-packages/xgboost/data.py:119: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


[15:47:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [15]:
preds

array([ 0.,  2.,  0., ..., 12., 12., 12.], dtype=float32)

#### Now, let's add some regression into the mix!

In [22]:
scores_list = []

for i in list(set(range(160)) - set(non_response_indeces)):
    data = data[:100, :]
    X, y = data[:, non_response_indeces], data[:, i]
    model = xgb.XGBRegressor()

    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import RepeatedKFold

    # define model evaluation method
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    scores_list.append(scores)
    # force scores to be positive
    # scores = absolute(scores)

KeyboardInterrupt: 

In [ ]:
scores

In [ ]:
3 in non_response_indeces

In [ ]:
scores_list